In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import ensemble, model_selection, preprocessing, tree

In [2]:
# Dataset contains NBA players stats from 1950 - 2017 (67 seasons)

nbaStats = pd.read_csv("/Users/ericleung/Desktop/Comp/Project/NBA (Data Science)/nba-players-stats/Seasons_Stats.csv")

In [3]:
# Let's take a look at our raw data

nbaStats

Unnamed: 0    Year             Player  Pos   Age   Tm     G    GS  \
0               0  1950.0    Curly Armstrong  G-F  31.0  FTW  63.0   NaN   
1               1  1950.0       Cliff Barker   SG  29.0  INO  49.0   NaN   
2               2  1950.0      Leo Barnhorst   SF  25.0  CHS  67.0   NaN   
3               3  1950.0         Ed Bartels    F  24.0  TOT  15.0   NaN   
4               4  1950.0         Ed Bartels    F  24.0  DNN  13.0   NaN   
5               5  1950.0         Ed Bartels    F  24.0  NYK   2.0   NaN   
6               6  1950.0        Ralph Beard    G  22.0  INO  60.0   NaN   
7               7  1950.0         Gene Berce  G-F  23.0  TRI   3.0   NaN   
8               8  1950.0      Charlie Black  F-C  28.0  TOT  65.0   NaN   
9               9  1950.0      Charlie Black  F-C  28.0  FTW  36.0   NaN   
10             10  1950.0      Charlie Black  F-C  28.0  AND  29.0   NaN   
11             11  1950.0        Nelson Bobb   PG  25.0  PHW  57.0   NaN   
12             12  1950.0    Jake Bornheimer  F-C  22.0  PHW  60.0   NaN   
13             13  1950.0       Vince Boryla   SF  22.0  NYK  59.0   NaN   
14             14  1950.0          Don Boven  F-G  24.0  WAT  62.0   NaN   
15             15  1950.0      Harry Boykoff    C  27.0  WAT  61.0   NaN   
16             16  1950.0        Joe Bradley    G  21.0  CHS  46.0   NaN   
17             17  1950.0        Bob Brannum   PF  24.0  SHE  59.0   NaN   
18             18  1950.0         Carl Braun  G-F  22.0  NYK  67.0   NaN   
19             19  1950.0      Frankie Brian    G  26.0  AND  64.0   NaN   
20             20  1950.0   Price Brookfield  F-G  29.0  ROC   7.0   NaN   
21             21  1950.0          Bob Brown    F  26.0  DNN  62.0   NaN   
22             22  1950.0         Jim Browne    C  20.0  DNN  31.0   NaN   
23             23  1950.0         Walt Budko   PF  24.0  BLB  66.0   NaN   
24             24  1950.0     Jack Burmaster    G  23.0  SHE  61.0   NaN   
25             25  1950.0       Tommy Byrnes  F-G  26.0  BLB  53.0   NaN   
26             26  1950.0       Bill Calhoun   SG  22.0  ROC  62.0   NaN   
27             27  1950.0        Don Carlson  G-F  30.0  MNL  57.0   NaN   
28             28  1950.0      Bob Carpenter  F-C  32.0  FTW  66.0   NaN   
29             29  1950.0        Jake Carter  F-C  25.0  TOT  24.0   NaN   
...           ...     ...                ...  ...   ...  ...   ...   ...   
24661       24661  2017.0     Deron Williams   PG  32.0  TOT  64.0  44.0   
24662       24662  2017.0     Deron Williams   PG  32.0  DAL  40.0  40.0   
24663       24663  2017.0     Deron Williams   PG  32.0  CLE  24.0   4.0   
24664       24664  2017.0   Derrick Williams   PF  25.0  TOT  50.0  11.0   
24665       24665  2017.0   Derrick Williams   PF  25.0  MIA  25.0  11.0   
24666       24666  2017.0   Derrick Williams   PF  25.0  CLE  25.0   0.0   
24667       24667  2017.0       Lou Williams   SG  30.0  TOT  81.0   1.0   
24668       24668  2017.0       Lou Williams   SG  30.0  LAL  58.0   1.0   
24669       24669  2017.0       Lou Williams   SG  30.0  HOU  23.0   0.0   
24670       24670  2017.0    Marvin Williams   PF  30.0  CHO  76.0  76.0   
24671       24671  2017.0    Reggie Williams   SF  30.0  NOP   6.0   0.0   
24672       24672  2017.0      Troy Williams   SF  22.0  TOT  30.0  16.0   
24673       24673  2017.0      Troy Williams   SF  22.0  MEM  24.0  13.0   
24674       24674  2017.0      Troy Williams   SF  22.0  HOU   6.0   3.0   
24675       24675  2017.0       Kyle Wiltjer   PF  24.0  HOU  14.0   0.0   
24676       24676  2017.0    Justise Winslow   SF  20.0  MIA  18.0  15.0   
24677       24677  2017.0        Jeff Withey    C  26.0  UTA  51.0   1.0   
24678       24678  2017.0     Christian Wood   PF  21.0  CHO  13.0   0.0   
24679       24679  2017.0        Metta World   SF  37.0  LAL  25.0   2.0   
24680       24680  2017.0     Brandan Wright   PF  29.0  MEM  28.0   5.0   
24681       24681  2017.0       Delon Wright   PG  24.0

In [4]:
#Since the NBA first adopted the 3 point line in 1979. We will drop the data where ['Year'] is less than 1979.

nbaStats_3ptAdded = nbaStats[nbaStats["Year"] >= 1979] 

In [5]:
# -------------------- Data Wrangling -------------------- #
# Filling all NaN inputs with 0
# Converting all the inputs of the coloumns 'Year', 'Age', 'G', and 'GS' to int.

nbaStats_3ptAdded = nbaStats_3ptAdded.fillna(0)
nbaStats_3ptAdded[['Year', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']] = nbaStats_3ptAdded[['Year', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']].astype(int) 


In [6]:
# Now we are done preprocessing our dataset. 

nbaStats_3ptAdded

Unnamed: 0  Year                Player Pos  Age   Tm   G  GS    MP  \
5382         5382  1979  Kareem Abdul-Jabbar*   C   31  LAL  80   0  3157   
5383         5383  1979         Tom Abernethy  PF   24  GSW  70   0  1219   
5384         5384  1979           Alvan Adams   C   24  PHO  77   0  2364   
5385         5385  1979          Lucius Allen  PG   31  KCK  31   0   413   
5386         5386  1979          Kim Anderson  SF   23  POR  21   0   224   
5387         5387  1979       Tiny Archibald*  PG   30  BOS  69   0  1662   
5388         5388  1979        Tate Armstrong  SG   23  CHI  26   0   259   
5389         5389  1979         Dennis Awtrey   C   30  TOT  63   0   746   
5390         5390  1979         Dennis Awtrey   C   30  BOS  23   0   247   
5391         5391  1979         Dennis Awtrey   C   30  SEA  40   0   499   
5392         5392  1979            Gus Bailey  SG   27  NOJ   2   0     9   
5393         5393  1979          Greg Ballard  SF   24  WSB  82   0  1552   
5394         5394  1979           Mike Bantom  PF   27  IND  81   0  2528   
5395         5395  1979            Tom Barker   C   23  TOT  39   0   476   
5396         5396  1979            Tom Barker   C   23  HOU   5   0    16   
5397         5397  1979            Tom Barker   C   23  BOS  12   0   131   
5398         5398  1979            Tom Barker   C   23  NYK  22   0   329   
5399         5399  1979         Marvin Barnes  PF   26  BOS  38   0   796   
5400         5400  1979           Rick Barry*  SF   34  HOU  80   0  2566   
5401         5401  1979           Tim Bassett  PF   27  NJN  82   0  1508   
5402         5402  1979           Butch Beard  PG   31  NYK   7   0    85   
5403         5403  1979           Ron Behagen  PF   28  TOT  15   0   165   
5404         5404  1979           Ron Behagen  PF   28  DET   1   0     1   
5405         5405  1979           Ron Behagen  PF   28  NYK   5   0    38   
5406         5406  1979           Ron Behagen  PF   28  KCK   9   0   126   
5407         5407  1979           Kent Benson   C   24  MIL  82   0  2132   
5408         5408  1979           Del Beshore  PG   22  MIL   1   0     1   
5409         5409  1979           Henry Bibby  PG   29  PHI  82   0  2538   
5410         5410  1979           Bob Bigelow  SF   25  SDC  29   0   413   
5411         5411  1979         Otis Birdsong  SG   23  KCK  82   0  2839   
...           ...   ...                   ...  ..  ...  ...  ..  ..   ...   
24661       24661  2017        Deron Williams  PG   32  TOT  64  44  1657   
24662       24662  2017        Deron Williams  PG   32  DAL  40  40  1171   
24663       24663  2017        Deron Williams  PG   32  CLE  24   4   486   
24664       24664  2017      Derrick Williams  PF   25  TOT  50  11   804   
24665       24665  2017      Derrick Williams  PF   25  MIA  25  11   377   
24666       24666  2017      Derrick Williams  PF   25  CLE  25   0   427   
24667       24667  2017          Lou Williams  SG   30  TOT  81   1  1994   
24668       24668  2017          Lou Williams  SG   30  LAL  58   1  1403   
24669       24669  2017          Lou Williams  SG   30  HOU  23   0   591   
24670       24670  2017       Marvin Williams  PF   30  CHO  76  76  2295   
24671       24671  2017       Reggie Williams  SF   30  NOP   6   0    79   
24672       24672  2017         Troy Williams  SF   22  TOT  30  16   557   
24673       24673  2017         Troy Williams  SF   22  MEM  24  13   418   
24674       24674  2017         Troy Williams  SF   22  HOU   6   3   139   
24675       24675  2017          Kyle Wiltjer  PF   24  HOU  14   0    44   
24676       24676  2017       Justise Winslow  SF   20  MIA  18  15   625   
24677       24677  2017           Jeff Withey   C   26  UTA  51   1   432   
24678       24678  2017        Christian Wood  PF   21  CHO  13   0   107   
24679       24679  2017           Metta World  SF   37  LAL  25   2   160   
24680       24680  2017        Brandan Wright  PF   29  MEM  28   5   447   
246

In [16]:
nbaStats_3ptAdded.set_index('Player')

Unnamed: 0  Year Pos  Age   Tm   G  GS    MP   PER  \
Player                                                                     
Kareem Abdul-Jabbar*        5382  1979   C   31  LAL  80   0  3157  25.5   
Tom Abernethy               5383  1979  PF   24  GSW  70   0  1219  13.6   
Alvan Adams                 5384  1979   C   24  PHO  77   0  2364  20.4   
Lucius Allen                5385  1979  PG   31  KCK  31   0   413   8.7   
Kim Anderson                5386  1979  SF   23  POR  21   0   224   3.0   
Tiny Archibald*             5387  1979  PG   30  BOS  69   0  1662  13.1   
Tate Armstrong              5388  1979  SG   23  CHI  26   0   259   6.5   
Dennis Awtrey               5389  1979   C   30  TOT  63   0   746   6.7   
Dennis Awtrey               5390  1979   C   30  BOS  23   0   247   5.4   
Dennis Awtrey               5391  1979   C   30  SEA  40   0   499   7.3   
Gus Bailey                  5392  1979  SG   27  NOJ   2   0     9   6.1   
Greg Ballard                5393  1979  SF   24  WSB  82   0  1552  14.9   
Mike Bantom                 5394  1979  PF   27  IND  81   0  2528  15.7   
Tom Barker                  5395  1979   C   23  TOT  39   0   476   9.1   
Tom Barker                  5396  1979   C   23  HOU   5   0    16  13.2   
Tom Barker                  5397  1979   C   23  BOS  12   0   131   9.5   
Tom Barker                  5398  1979   C   23  NYK  22   0   329   8.8   
Marvin Barnes               5399  1979  PF   26  BOS  38   0   796  12.3   
Rick Barry*                 5400  1979  SF   34  HOU  80   0  2566  14.7   
Tim Bassett                 5401  1979  PF   27  NJN  82   0  1508   8.0   
Butch Beard                 5402  1979  PG   31  NYK   7   0    85   9.0   
Ron Behagen                 5403  1979  PF   28  TOT  15   0   165   9.6   
Ron Behagen                 5404  1979  PF   28  DET   1   0     1 -60.3   
Ron Behagen                 5405  1979  PF   28  NYK   5   0    38   7.5   
Ron Behagen                 5406  1979  PF   28  KCK   9   0   126  10.8   
Kent Benson                 5407  1979   C   24  MIL  82   0  2132  17.3   
Del Beshore                 5408  1979  PG   22  MIL   1   0     1   0.0   
Henry Bibby                 5409  1979  PG   29  PHI  82   0  2538  11.2   
Bob Bigelow                 5410  1979  SF   25  SDC  29   0   413   6.0   
Otis Birdsong               5411  1979  SG   23  KCK  82   0  2839  18.4   
...                          ...   ...  ..  ...  ...  ..  ..   ...   ...   
Deron Williams             24661  2017  PG   32  TOT  64  44  1657  14.0   
Deron Williams             24662  2017  PG   32  DAL  40  40  1171  15.0   
Deron Williams             24663  2017  PG   32  CLE  24   4   486  11.4   
Derrick Williams           24664  2017  PF   25  TOT  50  11   804  10.6   
Derrick Williams           24665  2017  PF   25  MIA  25  11   377  10.1   
Derrick Williams           24666  2017  PF   25  CLE  25   0   427  11.1   
Lou Williams               24667  2017  SG   30  TOT  81   1  1994  21.4   
Lou Williams               24668  2017  SG   30  LAL  58   1  1403  23.9   
Lou Williams               24669  2017  SG   30  HOU  23   0   591  15.4   
Marvin Williams            24670  2017  PF   30  CHO  76  76  2295  13.7   
Reggie Williams            24671  2017  SF   30  NOP   6   0    79  11.7   
Troy Williams              24672  2017  SF   22  TOT  30  16   557   8.9   
Troy Williams              24673  2017  SF   22  MEM  24  13   418   7.6   
Troy Williams              24674  2017  SF   22  HOU   6   3   139  12.8   
Kyle Wiltjer               24675  2017  PF   24  HOU  14   0    44   6.7   
Justise Winslow            24676  2017  SF   20  MIA  18  15   625   8.2   
Jeff Withey                24677  2017   C   26  UTA  51   1   432  18.8   
Christian Wood             24678  2017  PF   21  CHO  13   0   107  15.1   
Metta World                24679  2017  SF   37  LAL  25   2   160   6.2   
Brandan Wright             24680  2017  PF   29  MEM  28   5   447  18

In [15]:
# nbaStats_3ptAdded['Player'][nbaStats_3ptAdded.BLK == nbaStats_3ptAdded.BLK.max()]

7638    Mark Eaton
Name: Player, dtype: object